In [ ]:
import httplib
import urllib
import base64
import json
import pandas as pd
import numpy as np
import requests

In [ ]:
# you have to sign up for an API key, which has some allowances. Check the API documentation for further details:
_url = 'https://api.projectoxford.ai/emotion/v1.0/recognizeInVideo'
_key = 'PRIMARY KEY' #Here you have to paste your primary key
_maxNumRetries = 10

In [ ]:
# URL direction: I hosted this on my domain

urlVideo = 'INSERT VIDEO URL HERE'

# Computer Vision parameters
paramsPost = { 'outputStyle' : 'perFrame'}

headersPost = dict()
headersPost['Ocp-Apim-Subscription-Key'] = _key
headersPost['Content-Type'] = 'application/json'

jsonPost = { 'url': urlVideo }

responsePost = requests.request( 'post', _url, json = jsonPost, data = None, headers = headersPost, params = paramsPost )

print responsePost.status_code

if responsePost.status_code == 202: # everything went well!
    videoIDLocation = responsePost.headers['Operation-Location']
    print videoIDLocation

In [ ]:
## Wait a bit, it's processing
headersGet = dict()
headersGet['Ocp-Apim-Subscription-Key'] = _key

jsonGet = {}
paramsGet = urllib.urlencode({})

getResponse = requests.request( 'get', videoIDLocation, json = jsonGet, data = None, headers = headersGet, params = paramsGet )


In [ ]:
#getResponse

In [ ]:
#jsonObject['error'] == {u'code': u'RateLimitExceeded', u'message': u'Rate limit is exceeded. Try again later.'}

In [ ]:
jsonObject

In [ ]:
import time

while True:
    time.sleep(20)
    getResponse = requests.request( 'get', videoIDLocation, json = jsonGet, data = None, headers = headersGet, params = paramsGet )
    
    jsonObject = json.loads(getResponse.text)
    #print jsonObject['progress']
    
    if hasattr(jsonObject, 'message'):
        print jsonObject['message']
    else:
        print jsonObject['progress']
                  
    if jsonObject['progress'] >= 100.0:
        break

In [ ]:
del(rawData)

In [ ]:
rawData = json.loads(json.loads(getResponse.text)['processingResult'])

In [ ]:
timeScale = rawData['timescale']
frameRate = rawData['framerate']

emotionPerFramePerFace = {}
currFrameNum = 0

for currFragment in rawData['fragments']:
    for currEvent in currFragment['events']:
        emotionPerFramePerFace[currFrameNum] = currEvent
        currFrameNum += 1

In [ ]:
with open('rawData.json', 'w') as f:
    json.dump(rawData, f)

In [ ]:
# Data collection
person  = []
for frame_no, v in emotionPerFramePerFace.copy().items():
    for i, minidict in enumerate(v):
        for k, v in minidict['scores'].items():
            minidict[k] = v
            
        minidict['frame'] = frame_no
        if i == 0:
            person.append(minidict)


df = pd.DataFrame(person)
#del df['scores']

# Saving in pd data-frame format:
df.to_csv("ONE SUBJECT DATA FILE.csv", index=False)